# Documentação: Coleta de Utilizadores (IDs) do GitHub por Localização

## Introdução
Este script recupera informações sobre o número total de utilizadores do GitHub por localização e obtém os IDs desses utilizadores de forma concorrente. Os dados são filtrados e salvos em um arquivo CSV.

## Dependências
Antes de executar o script, instale as bibliotecas necessárias:

```bash
pip install requests concurrent.futures pandas
```

## Estrutura do Script

### 1. Função `fetch_user_count`
```python
def fetch_user_count(location, session):
```
Essa função obtém o número total de utilizadores do GitHub para um determinado local utilizando a API do GitHub.

### 2. Função `get_total_users_by_location`
```python
def get_total_users_by_location(locations):
```
Essa função faz chamadas concorrentes para buscar o número de utilizadores do GitHub em diferentes localizações.

### 3. Lista de Localizações
```python
locations = ["Mozambique", "Moçambique", "Mocambique", "Maputo", "Matola", "Inhambane", "Manica", "Sofala", "Tete", "Zambezia", "Nampula", "Niassa", "Cabo Delgado"]
```
Define as localizações cujos utilizadores serão pesquisados.

### 4. Função `fetch_page`
```python
def fetch_page(location, page, session, headers):
```
Função auxiliar para obter uma página específica de IDs de utilizadores do GitHub.

### 5. Função `get_users_ids_by_location`
```python
def get_users_ids_by_location(location):
```
Essa função busca os IDs dos utilizadores do GitHub com base em uma localização especificada e trata a paginação da API.

### 6. Carregamento de IDs e Salvamento dos Dados
```python
dir_name = os.getenv('DIR_NAME')
base_path = f'../data/{dir_name}'
os.makedirs(base_path, exist_ok=True)
```
Define o diretório base onde os dados serão armazenados.

**Fluxo Completo:**
1. Coleta o número total de utilizadores para cada localização especificada.
2. Obtém os IDs dos utilizadores associados a cada localização.
3. Salva os IDs extraídos em um arquivo CSV:
   - `users_ids.csv`: Contém os IDs dos utilizadores.

### 7. Uso de Variáveis de Ambiente
O script requer duas variáveis de ambiente:
- `GITHUB_TOKEN`: Token de acesso à API do GitHub.
- `DIR_NAME`: Nome do diretório base para armazenamento dos dados processados.

Certifique-se de defini-las antes de executar o script.


# Main

In [1]:
import os
import time
import requests
import pandas as pd
from tqdm import tqdm
import concurrent.futures
from urllib.parse import urlparse, parse_qs

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def fetch_user_count(location, session):
    """
    Obtém o número total de utilizadores do GitHub para um determinado local usando a sessão fornecida.
    """

    token = os.getenv('GITHUB_TOKEN')
    headers = {
        'Authorization': f'Token {token}',
        'Accept': 'application/vnd.github.v3+json'
    }
    
    url = f'https://api.github.com/search/users?q=location:{location}'
    
    try:
        response = session.get(url, headers=headers)
        response.raise_for_status()
        total_count = response.json().get('total_count', 0)
        return {'total_users': total_count, 'location': location}
    except requests.RequestException as e:
        print(f"Erro na requisição ({location}): {e}")
        return {'total_users': None, 'location': location}

def get_total_users_by_location(locations):
    """
    Obtém o número total de utilizadores do GitHub para cada local especificado de forma concorrente.
    
    Args:
        locations (list): Lista de locais para obter o número de utilizadores.
        
    Returns:
        list: Lista de dicionários, onde cada dicionário contém o número total de utilizadores e o local correspondente.
    """
    users_by_location = []

    # Utilizar uma sessão para reutilização de conexões
    with requests.Session() as session:
        # Criar um pool de threads para obter os dados de forma concorrente
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            # Agendar todas as tarefas de obtenção e preservar a ordem recolhendo os futures numa lista.
            futures = [executor.submit(fetch_user_count, location, session) for location in locations]
            # Obter os resultados na mesma ordem da lista de locais
            users_by_location = [future.result() for future in futures]

    return users_by_location

# Lista de locais para obter o número de utilizadores
# A província de Gaza foi omitida para evitar confusão com regiões com nomes parecidos.
# Outros nomes que possam causar ambiguidades também serão excluídos.
locations = [
    'Mozambique', 
    'Moçambique', 
    'Mocambique', 
    'Maputo', 
    'Matola',
    'Inhambane', 
    'Manica',
    'Sofala',
    'Tete',
    'Zambezia',
    'Nampula',
    'Niassa',
    'Cabo Delgado'
]

if __name__ == "__main__":
    results = get_total_users_by_location(locations)

    # Exibir os resultados
    print("Usuários por localização \n")
    for result in results:
        print(f"{result['location']}: {result['total_users']}")


Usuários por localização 

Mozambique: 1803
Moçambique: 382
Mocambique: 54
Maputo: 1289
Matola: 74
Inhambane: 19
Manica: 8
Sofala: 46
Tete: 48
Zambezia: 13
Nampula: 85
Niassa: 13
Cabo Delgado: 3


In [3]:
def fetch_page(location, page, session, headers):
    """
    Função auxiliar para obter uma página específica de IDs de utilizadores do GitHub.
    """
    url = "https://api.github.com/search/users"
    params = {
        'q': f'location:{location}',
        'per_page': 100,
        'page': page,
        'order': 'asc',
        'sort': 'joined'
    }
    response = session.get(url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()
    return [user["id"] for user in data.get("items", [])]

def get_users_ids_by_location(location):
    """
    Obtém os IDs de utilizadores da API do GitHub com base no local especificado.

    Args:
        location (str): Local dos utilizadores a procurar.

    Returns:
        list: Lista de IDs de utilizadores encontrados ou None se a requisição falhar.
    """
    token = os.getenv('GITHUB_TOKEN')

    if not token:
        print("Erro: O token de autenticação (GITHUB_TOKEN) não foi encontrado. Certifique-se de que está definido corretamente.")
        return None
            
    headers = {
        'Authorization': f'Token {token}',
        'Accept': 'application/vnd.github.v3+json'
    }

    base_url = "https://api.github.com/search/users"
    params = {
        'q': f'location:{location}',
        'per_page': 100,  # Número máximo de itens por página
        'page': 1,        # Começa na página 1
        'order': 'asc',
        'sort': 'joined',
    }

    user_ids = []

    with requests.Session() as session:
        # Obter a primeira página
        response = session.get(base_url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Erro na requisição ({location}): {response.status_code}")
            return None

        data = response.json()
        user_ids.extend([user["id"] for user in data.get("items", [])])

        # Verificar se há páginas adicionais através do link 'last'
        if 'last' in response.links:
            # Extrair o número total de páginas do URL do link 'last'
            last_url = response.links['last']['url']
            parsed_url = urlparse(last_url)
            query_params = parse_qs(parsed_url.query)
            last_page = int(query_params.get('page', [1])[0])

            # Agendar requisições concorrentes para as páginas 2 até last_page
            pages = range(2, last_page + 1)
            with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
                futures = [
                    executor.submit(fetch_page, location, page, session, headers)
                    for page in pages
                ]
                for future in concurrent.futures.as_completed(futures):
                    try:
                        ids = future.result()
                        user_ids.extend(ids)
                    except Exception as exc:
                        print(f"Erro ao obter uma página para {location}: {exc}")

    return user_ids or None


In [5]:
dir_name = os.getenv('DIR_NAME')
base_path = f'../data/{dir_name}'
os.makedirs(base_path, exist_ok=True)

data = []

for loc in tqdm(locations, desc="Processando localizações"):
    final_data = get_users_ids_by_location(loc)
    
    if final_data:
        for user_id in final_data:
            data.append({'user_id': user_id})
    
    # Aguarda 10 segundos após processar cada localização para não exceder o limite de 30 requisições por minuto
    time.sleep(10)


Processando localizações: 100%|██████████| 13/13 [02:29<00:00, 11.47s/it]


In [6]:
df = pd.DataFrame(data, columns=['user_id']).drop_duplicates().sort_values(by='user_id')

file_name = f'{base_path}/users_ids.csv'
df.to_csv(file_name, index=False)

print(f"Gravados IDs de {len(df)} usuários.")

Gravados IDs de 2029 usuários.
